# Criando mapas interativos a partir de um GeoDataFrame

## 1. Mostrar os dados no mapa

Para criar os mapas e construir a visualização, vamos utilizar a biblioteca `Folium`

In [7]:
# Importando as bibliotecas GeoPandas e Folium
import geopandas as gpd
import folium.map
import pandas as pd

In [2]:
# Criando um mapa simples usando o folium
fmap = folium.Map()

fmap

Com apenas uma função já conseguimos criar o mapa. Nele nós conseguimos realizar as operações de **zoom** e **movimento com mouse.**

Agora, vamos aplicar os nossos dados a esse mapa.

In [3]:
# Carregando os dados de roubo
gdf_roubos_sp = gpd.read_file('data/roubo_celular_capital.json', driver='GeoJSON')

# Carregando os dados de geometria
gdf_geometria_sp = gpd.read_file('data/capital_são_paulo.json', driver='GeoJSON')

In [4]:
gdf_geometria_sp

,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry
0,3550308,São Paulo,SP,1521.202,"POLYGON ((-46.63512 -23.83850, -46.63455 -23.8..."


### Vamos criar um mapa e mostrar os dados extraídos

In [8]:
import folium.features

gdf_roubos_slice = gdf_roubos_sp.iloc[:100]

# Converta todas as colunas de tipo Timestamp para string
for col in gdf_roubos_slice.columns:
    if pd.api.types.is_datetime64_any_dtype(gdf_roubos_slice[col]):
        gdf_roubos_slice[col] = gdf_roubos_slice[col].astype(str)

# Após a conversão, você pode tentar novamente adicionar o GeoJson ao mapa
fmap = folium.Map()

# Adiciona a camada GeoJSON ao mapa
feat_geojson = folium.features.GeoJson(gdf_roubos_slice)
fmap.add_child(feat_geojson)

# Exibe o mapa
fmap


c:\Users\cleit\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\cleit\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\cleit\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

### Os dados foram extraídos corretamente, mas foi necessário fazer um zoom no mapa. Vamos deixar o mapa mais responsivo.

In [9]:
# Definindo ponto de início de renderização do mapa
media_latitude = gdf_roubos_slice['LATITUDE'].mean()
media_longitude = gdf_roubos_slice['LONGITUDE'].mean()

fmap = folium.Map(location=[media_latitude, media_longitude])

feat_geojson = folium.features.GeoJson(gdf_roubos_slice)

fmap.add_child(feat_geojson)

fmap

## 2. Criar mapas de agrupamentos (*clusters*)

No mapa acima, utilizamos somente uma fração dos dados extraídos do nosso GeoDataFrame, e para mostrar todos os dados? Ao adicionar uma grande quantidade de dados no nosso mapa, pode causar dificuldade na visualização, por isso vamos utilizar um plugin do `Folium` para melhorar a visualização, fazendo agrupamentos dinâmicos.

In [10]:
# Importando o plugin FastMarkerCluster
from folium.plugins import FastMarkerCluster

In [11]:
fmap = folium.Map(location=[media_latitude, media_longitude])

mc = FastMarkerCluster(gdf_roubos_sp[['LATITUDE', 'LONGITUDE']])

fmap.add_child(mc)

fmap